<a href="https://colab.research.google.com/github/AndrewstheBuilder/FromScratch_NeuralNetworks/blob/main/Micrograd_MNIST.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### MNIST using micrograd_andrews module

In [1]:
pip install micrograd_andrews

In [2]:
from micrograd_andrews.engine import Value
from micrograd_andrews.nn import Neuron, Layer, MLP